# DeepFool e Robustez dos Modelos

O DeepFool é um algoritmo usado para encontrar a menor perturbação adversária capaz de enganar um classificador baseado em redes neuronais.<br>
O objetivo é modificar ligeiramente uma entrada classificada corretamente de forma que ela seja reclassificada para uma classe diferente. O processo é iterativo, ajustando a entrada gradualmente até que ela ultrapasse a fronteira de decisão da classe original.<br>
Esse processo é amplamente utilizado para estudar a robustez dos modelos contra ataques adversários.

Temos o seu funcionamento:

- **Inicialização**: O algoritmo começa com uma entrada classificada corretamente e busca uma pequena alteração, chamada de perturbação, para movê-la para outra classe.

- **Iterativo**: A cada iteração, o DeepFool lineariza a função de decisão do classificador em torno da entrada, aproximando as fronteiras de decisão entre classes com planos.

- **Perturbação Mínima**: A perturbação necessária para cruzar a fronteira é calculada com base na direção da fronteira de decisão mais próxima, que é determinada pelo gradiente da função de decisão do modelo.

- **Atualização**: A entrada é então atualizada para a nova versão perturbada, e o processo continua até que a entrada seja classificada de forma diferente.

- **Saída:** O algoritmo retorna a perturbação mínima encontrada e a entrada adversária resultante, que foi modificada para enganar o classificador.

# 

# 0. Libraries

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import models, layers, regularizers, optimizers
from tensorflow.python.client import device_lib
import pandas as pd
import numpy as np
import pickle
import os
import librosa
from copy import deepcopy

2024-12-03 16:47:54.884187: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-03 16:47:54.890758: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-03 16:47:54.978733: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-03 16:47:55.048588: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733244475.108241    3412 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733244475.12

# 

# 1. Implementação do DeepFool

Iremos implementar agora todas as funções necessárias ao funcionamento do DeepFool.

In [10]:
#função que calcula o gradiente do k-ésimo elemento à saída do modelo, dada uma entrada x, em relação à entrada x
def get_gradient(model, x, k):

    #calcula o k-ésimo valor da saída do modelo usando x como entrada sob a vigilância do GradientTape do tensorflow
    with tf.GradientTape(persistent=True) as tape:
        inputs = [tf.cast(input_value, dtype=tf.float64) for input_value in x]
        for input_value in inputs:
            tape.watch(input_value)
        results = model(inputs)
        results_k = results[0,k]

    #obter o gradiente do k-ésimo elemento à saída do modelo
    gradients = tape.gradient(results_k, inputs)
    del tape
    return [grad.numpy() for grad in gradients], results

In [ ]:
#implementação do DeepFool, com o modelo a usar no algoritmo, x0 entrada inicial sem perturbação, eta valor de overshoot a ser multiplicado pela perturbação de cada iteração, max_iter numero maximo de itercações
def deepfool(model, x0, eta=0.01, max_iter=20):

    #obtém a label estimada inicial
    f_x0 = model(x0).numpy().flatten()
    label_x0 = f_x0.argsort()[::-1][0]

    loop_i = 0
    xi = deepcopy(x0)
    label_xi = label_x0
    r = []

    #main loop
    while label_xi == label_x0 and loop_i < max_iter:
        w_l = [np.zeros(x_input.shape) for x_input in x0]
        f_l = 0
        fk_wk_min = np.inf
        grad_f_label_x0_on_xi, f_xi = get_gradient(model, xi, label_x0)

        for k in range(10): # k = 0, ..., 9 (possible classes in the problem considered for this project)
            if (k == label_x0):
                continue
            grad_f_k_on_xi, f_xi = get_gradient(model, xi, k)
            w_k = [g_f_k - g_f_label for g_f_k, g_f_label in zip(grad_f_k_on_xi, grad_f_label_x0_on_xi)]
            w_k_norm = np.sqrt(np.sum(np.fromiter([np.linalg.norm(w_k_input)**2 for w_k_input in w_k], dtype=np.float32)))
            f_k = f_xi[0,k] - f_xi[0,label_x0]
            fk_wk = np.linalg.norm(f_k) / (w_k_norm + 1e-3)
            if fk_wk < fk_wk_min:
                w_l, f_l = w_k, f_k

        w_l_squared_norm = np.sum(np.fromiter([np.linalg.norm(w_l_input)**2 for w_l_input in w_l], dtype=np.float32))
        f_l_norm = np.linalg.norm(f_l)
        ri_const = f_l_norm / (w_l_squared_norm + 1e-3)
        ri = [ri_const * w_l_input for w_l_input in w_l]
        r.append(ri)
        xi_new = [xi_item + (1+eta)*ri_item for xi_item, ri_item in zip(xi, ri)]
        xi = xi_new
        label_xi = model(xi).numpy().flatten().argsort()[::-1][0]
        loop_i += 1

    #main loop acabado
    r_sum = [np.zeros(x_input.shape) for x_input in x0]
    for i in range(len(x0)):
        for r_i in r:
            r_sum[i] += r_i[i][0]

    #devolve o valor de r(x), o número de iterações realizadas e o novo rótulo obtido adicionando a perturbação à entrada
    return r_sum, loop_i, label_xi

In [ ]:
#funcao para calcular o valor de ||r(x)|| / ||x||
def example_robustness(x, r):
    r_norm = np.sqrt(np.sum(np.fromiter([np.linalg.norm(r_input)**2 for r_input in r], dtype=np.float32)))
    x_norm = np.sqrt(np.sum(np.fromiter([np.linalg.norm(x_input)**2 for x_input in x], dtype=np.float32)))
    return r_norm / x_norm

In [ ]:
#função para devolver a robustez de um modelo
def model_robustness(example_robustness_list):
    mean = np.mean(np.array(example_robustness_list))
    std = np.std(np.array(example_robustness_list))
    return mean, std

# 

# 2. DeepFool vs MLP

Este código executa uma avaliação da robustez de um modelo de rede neural MLP (Perceptron Multicamadas) por meio da divisão dos dados em pastas de validação cruzada. O código segue as seguintes fases para cada pasta:

- **Transferência de dados**: As informações do fold atual são obtidas do DataFrame, levando em conta várias representações de áudio, como o espectrograma mel, o cromagrama e as características espectrais.

- **Preparação do modelo**: É carregado e compilado o modelo MLP previamente treinado para ser utilizado na análise.

- E para cada exemplo presente no conjunto de dados do folder em curso:

    - As **informações são estruturadas** como uma série de 6 atributos de entrada, que correspondem às representações obtidas dos dados.

    - Por fim o **DeepFool é utilizado** para determinar o menor distúrbio adverso necessário para modificar a previsão do modelo. A robustez do exemplo é calculada como a relação entre a norma da perturbação adversarial e a norma do exemplo original.
    
- **Armazenamento dos resultados**: Os valores de robustez de cada exemplo são salvos em um arquivo para posterior análise.

Este procedimento é repetido para todos os folds, permitindo avaliar a robustez do modelo em diferentes subconjuntos dos dados.


In [12]:
with open("/content/extracoes_mlp.pkl", "rb") as dados:
        df_data = pickle.load(dados)
    dados.close()
    
df_data.head()

,fold,melspectrogram,chromagram,spectral_centroid,spectral_bandwidth,spectral_flatness,spectral_rolloff,label
0,fold10,"[0.3427932185914244, 0.4597565102867449, 0.566...","[0.024030005965636037, 0.022884723327388, 0.02...","[1.0, 0.26519852242376013, 0.279489527736959, ...","[1.0, 0.586902507704785, 0.5950458954717306, 0...","[0.9999999999999999, 0.0011557663016491656, 0....","[1.0, 0.2145922746781116, 0.24678111587982834,...","[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,fold10,"[0.4019990178950694, 0.5387167979572665, 0.657...","[0.0, 0.0025119146556220606, 0.011815343070643...","[0.9999999999999999, 0.5714512204257515, 0.517...","[1.0, 0.6083331850877384, 0.6508332225580915, ...","[0.9999999999999999, 0.05145324220792339, 0.01...","[1.0, 0.4679334916864608, 0.4560570071258907, ...","[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,fold10,"[0.3585990476035814, 0.5075671434805749, 0.643...","[0.05072147436540447, 0.04892150092399211, 0.0...","[1.0, 0.557251711998936, 0.6818391053701994, 0...","[1.0, 0.7573797645773904, 0.6976837713584626, ...","[0.9999999999999998, 0.1334785083420296, 0.007...","[1.0, 0.5631067961165048, 0.5606796116504854, ...","[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,fold10,"[0.4696441409528247, 0.6035035756567209, 0.719...","[0.1646298514503709, 0.1573688786873304, 0.156...","[0.8333398745657035, 0.8081148109215915, 0.889...","[0.850667058692703, 0.8312376687144649, 0.8610...","[0.051290638035947855, 0.0006129429235882337, ...","[0.7335526315789475, 0.7105263157894738, 0.769...","[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,fold10,"[0.4043041831837222, 0.5495159766371622, 0.676...","[0.0683368244764111, 0.07745417347432648, 0.08...","[0.5945901624436382, 0.48611240925922655, 0.61...","[1.0, 0.8729978479840884, 0.9858855910775775, ...","[0.11006144775969313, 0.026223634144737374, 0....","[0.8605263157894736, 0.6894736842105262, 0.857...","[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [13]:
# Definindo o loop para os folds
for f in range(1, 10 + 1):

    # Lista para armazenar os valores de robustez
    robustness_values_mlp_fold = []

    # Utilizar apenas os dados do f-ésimo fold
    X_fold = df_data[df_data['fold'] == f"fold{f}"]

    # Preparar as entradas para o modelo MLP
    X_mel = np.asarray(X_fold["melspectrogram"].to_list()).astype(np.float32)
    X_chroma = np.asarray(X_fold["chromagram"].to_list()).astype(np.float32)
    centroid = np.asarray(X_fold["spectral_centroid"].to_list()).astype(np.float32)
    bandwidth = np.asarray(X_fold["spectral_bandwidth"].to_list()).astype(np.float32)
    flatness = np.asarray(X_fold["spectral_flatness"].to_list()).astype(np.float32)
    rolloff = np.asarray(X_fold["spectral_rolloff"].to_list()).astype(np.float32)

    # Carregar o modelo MLP
    fold_model_mlp = keras.models.load_model("/content/modelo_mlp.h5", compile=False)
    fold_model_mlp.compile(
        optimizer=optimizers.Adam(learning_rate=0.001),
        loss="categorical_crossentropy",
        metrics=["accuracy"]
    )

    # Rodar o DeepFool para cada exemplo no fold
    for i in range(len(X_fold)):

        # Criar as entradas de maneira que o modelo receba 6 entradas
        example_input = [
            X_mel[i:i+1],        # Mel-spectrogram
            X_chroma[i:i+1],     # Chromagram
            centroid[i:i+1],     # Spectral centroid
            bandwidth[i:i+1],    # Spectral bandwidth
            flatness[i:i+1],     # Spectral flatness
            rolloff[i:i+1]       # Spectral rolloff
        ]

        # Rodar o DeepFool para o exemplo atual
        perturbation, iters, fool_label = deepfool(fold_model_mlp, example_input, eta=1e6)

        # Calcular o valor de robustez ||r(x)|| / ||x|| e adicionar à lista
        robustness_value = example_robustness(example_input, perturbation)
        robustness_values_mlp_fold.append(robustness_value)

    # Salvar os resultados de robustez para o fold
    with open(f"/content/robustness_values_mlp_fold{f}.pkl", "wb") as dados:
        pickle.dump(robustness_values_mlp_fold, dados)
    dados.close()

In [15]:
# verifica os resultados dos modelos de cada fold(pasta)
for f in range(1, 10+1):
    
    with open(f"/content/robustness_values_mlp_fold{f}.pkl", "rb") as dados:
        robustness_values_mlp_fold = pickle.load(dados)
    dados.close()
    
    mean_robustness_mlp, std_robustness_mlp = model_robustness(robustness_values_mlp_fold)
    print(f"Fold {f} - The MLP model has a robustness of {mean_robustness_mlp: .7f} +/- {std_robustness_mlp: .7f}.")

Fold 1 - The MLP model has a robustness of  0.0205068 +/-  0.0103255.
Fold 2 - The MLP model has a robustness of  0.0203518 +/-  0.0087716.
Fold 3 - The MLP model has a robustness of  0.0230387 +/-  0.0092817.
Fold 4 - The MLP model has a robustness of  0.0218645 +/-  0.0097596.
Fold 5 - The MLP model has a robustness of  0.0211337 +/-  0.0099278.
Fold 6 - The MLP model has a robustness of  0.0226863 +/-  0.0091783.
Fold 7 - The MLP model has a robustness of  0.0207152 +/-  0.0086542.
Fold 8 - The MLP model has a robustness of  0.0211000 +/-  0.0089398.
Fold 9 - The MLP model has a robustness of  0.0189192 +/-  0.0080271.
Fold 10 - The MLP model has a robustness of  0.0190777 +/-  0.0079830.


# 

# 3. Conclusão

O modelo MLP apresenta uma robustez adversarial relativamente uniforme, mas com um valor médio baixo. Isso indica que ele pode ser facilmente enganado por pequenas perturbações adversariais.<br>
Para melhorar a robustez, podiamos explorar modelos mais complexos, como Redes Neurais Convolucionais (CNNs), infelizmente devido ao tempo eu nao consegui executar o deepfool para cnn.